In [4]:
import os
import pathlib
import json

import pandas as pd

In [ ]:
BASE_DIR = str(pathlib.Path().resolve())
TARGET_DIR = ''
WORKING_DIR = f'{BASE_DIR}/{TARGET_DIR}'

dir_list = os.listdir(WORKING_DIR)

mapping = ''
mapping_dict = {}

i = 1

filtered_files = [filename for filename in dir_list if filename.endswith('.xlsx')]
mapping_dict = {number + 1: filename for number, filename in enumerate(filtered_files)}

#
# print(f'BASE_DIR: {BASE_DIR}')
# print(f'TARGET_DIR: {TARGET_DIR}')
# print(f'WORKING_DIR: {WORKING_DIR}')
print(f'dir_list: {dir_list}')

if not mapping_dict:
    print('\n==================================================')
    print(f'{"!В папке отсутствует mapping файл формата xlsx!":^50}')
    print('==================================================')
elif len(mapping_dict) == 1:
    mapping = next(iter(mapping_dict.values()))
elif len(mapping_dict) > 1:
    for key in mapping_dict.keys():
        print(key,': ', mapping_dict[key])
        
    mapping_key = input('\nWrite mapping number: ')

    mapping = mapping_dict[int(mapping_key)]

print(f'mapping: {mapping}')

csv = '98_count_all_all_tables.csv'

In [ ]:
df = pd.read_csv(f'{WORKING_DIR}/{csv}', header=0, delimiter=';')
df

In [ ]:
df = df.sort_values('cnt')
df

In [ ]:
zero_table_df = df[df['cnt']==0]
print(zero_table_df.count())
zero_table_lst = zero_table_df['TBL'].unique()
print(zero_table_lst.tolist())

In [ ]:
next_df = df[df['cnt']!=0].copy()
print(next_df.head())
print(next_df.count())
next_table_lst = next_df['TBL'].unique()
print(next_table_lst.tolist())

In [ ]:
next_df.index = range(1, len(next_df) + 1)
next_df

In [ ]:
next_0_100_df = next_df[next_df['cnt']<101]
# next_0_100_df.to_csv(f'{WORKING_DIR}/98_0_100_tables.csv')
next_0_100_df['TBL'].unique()

In [ ]:
next_101_10000_df = next_df[next_df['cnt'].isin(range(101,10000))]
next_101_10000_df.to_csv(f'{WORKING_DIR}/98_101_10000_tables.csv')
next_101_10000_df['TBL'].unique()

In [ ]:
next_10001_1000000_df = next_df[next_df['cnt'].isin(range(10001,1000000))]
next_10001_1000000_df.to_csv(f'{WORKING_DIR}/98_10001_1000000_tables.csv')
next_10001_1000000_df['TBL'].unique()

In [ ]:
next_1000000_end_df = next_df[next_df['cnt']>1000000]
next_1000000_end_df.to_csv(f'{WORKING_DIR}/98_1000001_end_tables.csv')
next_1000000_end_df['TBL'].unique()

In [ ]:
df = pd.read_excel(f'{WORKING_DIR}/{mapping}', sheet_name='Mapping',
                   usecols="D,T:V,Z,AA")

df = df.drop(0,axis=0)

df.columns = ['SchemaS', 'SchemaT', 'Table', 'Code', 'Data Type', 'Length']

print(df.count())

df['Code'] = df['Code'].apply(lambda x: str(x).strip())

df = df[df['Code']!='hdp_processed_dttm']
print(df.count())
df.head()

In [ ]:
df = pd.read_excel(f'{WORKING_DIR}/{mapping}', sheet_name='Mapping',
                   usecols="D,T:V,Z,AA")

df = df.drop(0,axis=0)

df.columns = ['SchemaS', 'SchemaT', 'Table', 'Code', 'Data Type', 'Length']

df = df[df['Code']!='hdp_processed_dttm']

df = df.sort_values(['Table'])

df.index = range(1, len(df) + 1)

df.head()

In [ ]:
tables_lst = df['Table'].unique()
print(
    len(tables_lst)
)
print(tables_lst)

In [ ]:
test_df = df[df['Table']=='abbreviations_ref'].copy()
test_df = test_df.fillna('')
test_df

In [ ]:
test_row = test_df.iloc[0]
schema_s = test_row['SchemaS']
print(schema_s)
schema_t = test_row['SchemaT']
print(schema_t)
table = test_row['Table']
print(table)
columns = []
columns_casts = []

for _, row in test_df.iterrows():
    columns.append(row['Code'])
    columns_casts.append(
        {
            "name": row['Code'],
            "colType": row['Data Type']
        }
    )

print(columns)
print(columns_casts)

In [ ]:
test_row = test_df.iloc[0]
schema_s = test_row['SchemaS']
print(schema_s)
schema_t = test_row['SchemaT']
print(schema_t)
table = test_row['Table']
print(table)

query_full = ''
query_prefix = 'select '
query_suffix = ' from $schema.$table'
query_cast_list = []

for _, row in test_df.iterrows():
    attr = row['Code']
    typ = row['Data Type']
    length = f"({test_row['Length']})" if test_row['Length'] else ''
    query_cast_list.append(f"cast([{attr}] as {typ}{length} ) as '{attr}'")

query_full = ', '.join(query_cast_list)

query_full = query_prefix + query_full + query_suffix

print(query_full)

In [ ]:
test_flow_entity_lst = []

template1 = {
    "loadType": "Scd1Replace",
    "source": {
        "schema": schema_s,
        "table": table,
        "columns": columns,
        "columnCasts": columns_casts,
        "jdbcDialect": "..."
    },
    "target": {
        "table": table
    }
}

test_flow_entity_lst.append(template1)

main_json_template = {
  "connection": {
    "connType": "jdbc",
    "url": "...",
    "driver": "...",
    "user": "...",
    "password": "..."
  },
  "commonInfo": {
    "targetSchema": schema_t,
    "etlSchema": schema_t,
    "logsTable": "logs597"
  },
  "flows": test_flow_entity_lst
}

res_json = json.dumps(main_json_template)

print(
    res_json.replace('}}', '} }').replace('{{', '{ {')\
            .replace('}]', '} ]').replace('[{', '[ {')\
            .replace(']}', '] }').replace('{[', '{ [')\
            .replace('"}', '" }').replace('{"', '{ "')
    )

In [ ]:
def print_flow_entity(schema_s, table, columns, columns_casts):
	template = {
		"loadType": "Scd1Replace",
		"source": {
			"schema": schema_s,
			"table": table,
			"columns": columns,
			"columnCasts": columns_casts,
			"jdbcDialect": "..."
		},
		"target": {
			"table": table
		}
	}

	return json.dumps(template)

json_str = print_flow_entity(schema_s, table, columns, columns_casts)

json_str